In [1]:
import pandas as pd
import time
from sqlalchemy import create_engine
import numpy as np
from polygon import RESTClient

In [2]:
#creating api key variable as Client
client = RESTClient(api_key="enter polygon api key")

# Extract

In [3]:
#Using polygon RESTClient endpoint to get aggregate bars from 1-1-2023 to 2024-01-01
def fetch_stock_data(tick, multiplier, timespan, from_, to):
    data = client.get_aggs(tick, multiplier, timespan, from_, to)
    
    if data != None:
        stock_data = pd.DataFrame(data)
        return stock_data
    else:
        print(f'Error fetching data for {tick}')
        return None

In [4]:
#setting up variables and calling the function to get aggregate bars (fetch_stock_data)
#We are calling for 6 stocks, the api only allows 5 calls per minute.
#time.sleep allows us to call the api without reaching the limit
initial_invest = [14382, 23789, 10045, 12673, 16461, 22650]
ticker = ['SONY', 'MSFT', 'NVDA', 'F', 'TSLA','VT']
multiplier = 1
timespan = 'day'
from_ = '2023-01-01'
to = '2024-01-01'
stock_data_to_concat = []
total_return = []
stock_return = []
cumulative_annual_return = []
volatility = []
ten_day = []
hundred_day = []
sharpe = []

for tick in ticker:
    stock_data = fetch_stock_data(tick, multiplier, timespan, from_, to)
    stock_data.insert(0, 'stock', tick)
    stock_data_to_concat.append(stock_data)
    time.sleep(11)

#calculating stock metrics
for i, dframe in enumerate(stock_data_to_concat):
    
    #calculating total return on stock & portfolio
    total_return_cal = ((dframe['close'].iloc[-1] - dframe['close'].iloc[0]))
    stock_return.append(total_return_cal)
    total_return.append(total_return_cal * (initial_invest[i] / dframe['close'].iloc[0]))

    #calculating cumulative/annual return on stock & portfolio
    cumulative_annual_return.append((total_return_cal / dframe['close'].iloc[0]) * 100)

    #calculating volatility on stock
    vol_cal = dframe['close'].std() * np.sqrt(dframe['close'].count())
    volatility.append(vol_cal)

    #calcuating 10 day / 100 day averages
    ten_day = dframe['close'].tail(10).sum() / 10
    hundred_day = dframe['close'].tail(100).sum() / 100

    #calculating sharpe ratio
    sharpe_cal = (((dframe['close'].pct_change().dropna().mean() * dframe['close'].count()) - .03) / vol_cal) * 100
    sharpe.append(sharpe_cal)
    
stock_data = pd.concat(stock_data_to_concat, ignore_index=True)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [61]:
#creating portfolio metrics dataframe
portfolio_metrics = pd.DataFrame(columns=[
    'stock',
    'initial_investment',
    'total_return',
    'cumulative_return',
    'annual_return',
    'volatility',
    'sharpe_ratio'
])
portfolio_metrics['stock'] = ticker
portfolio_metrics['initial_investment'] = initial_invest
portfolio_metrics['total_return'] = [round(num, 2) for num in total_return]
portfolio_metrics['cumulative_return'] = [round(num, 2) for num in cumulative_annual_return]
portfolio_metrics['annual_return'] = [round(num, 2) for num in cumulative_annual_return]
portfolio_metrics['volatility'] = [round(num, 2) for num in volatility]
portfolio_metrics['sharpe_ratio'] = sharpe

#creating stock metrics dataframe
stock_metrics = pd.DataFrame(columns=[
    'stock',
    'total_return',
    'cumulative_return',
    'volatility',
    'ten_day_average',
	'hundred_day_average',
    'sharpe_ratio'
])
stock_metrics['stock'] = ticker
stock_metrics['total_return'] = [round(num, 2) for num in stock_return]
stock_metrics['cumulative_return'] = [round(num, 2) for num in cumulative_annual_return]
stock_metrics['volatility'] = [round(num, 2) for num in volatility]
stock_metrics['ten_day_average'] = round(ten_day, 2)
stock_metrics['hundred_day_average'] = round(hundred_day, 2)
stock_metrics['sharpe_ratio'] = sharpe

# Transform

In [62]:
portfolio_metrics.head(6)

,stock,initial_investment,total_return,cumulative_return,annual_return,volatility,sharpe_ratio
0,SONY,14382,3384.88,23.54,23.54,76.23,0.280095
1,MSFT,23789,13549.74,56.96,56.96,647.89,0.070084
2,NVDA,10045,24705.16,245.94,245.94,170.07,0.782016
3,F,12673,553.36,4.37,4.37,18.70,0.411909
4,TSLA,16461,21376.46,129.86,129.86,662.76,0.142310
5,VT,22650,4370.32,19.29,19.29,57.50,0.270098


In [63]:
stock_metrics.head(6)

,stock,total_return,cumulative_return,volatility,ten_day_average,hundred_day_average,sharpe_ratio
0,SONY,18.04,23.54,76.23,102.22,96.16,0.280095
1,MSFT,136.46,56.96,647.89,102.22,96.16,0.070084
2,NVDA,35.21,245.94,170.07,102.22,96.16,0.782016
3,F,0.51,4.37,18.70,102.22,96.16,0.411909
4,TSLA,140.38,129.86,662.76,102.22,96.16,0.142310
5,VT,16.64,19.29,57.50,102.22,96.16,0.270098


In [64]:
stock_data.head()

,stock,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,SONY,77.15,77.680,76.39,76.65,660630.0,76.7866,1672722000000,13809,None
1,SONY,77.99,78.390,77.49,77.80,596412.0,77.9470,1672808400000,13577,None
2,SONY,77.78,78.115,77.31,77.50,540993.0,77.6747,1672894800000,12203,None
3,SONY,79.23,81.640,79.10,81.39,896688.0,80.8119,1672981200000,22789,None
4,SONY,81.68,83.340,81.63,82.62,918041.0,82.7863,1673240400000,17973,None


In [65]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stock         1500 non-null   object 
 1   open          1500 non-null   float64
 2   high          1500 non-null   float64
 3   low           1500 non-null   float64
 4   close         1500 non-null   float64
 5   volume        1500 non-null   float64
 6   vwap          1500 non-null   float64
 7   timestamp     1500 non-null   int64  
 8   transactions  1500 non-null   int64  
 9   otc           0 non-null      object 
dtypes: float64(6), int64(2), object(2)
memory usage: 117.3+ KB


In [66]:
#timestamp is in unix, I'm converting to datetime using Milliseconds 'ms' units
stock_data['timestamp'] = pd.to_datetime(stock_data['timestamp'], unit='ms')

In [67]:
#removing otc (over-the-counter) column as it wont be needed
stock_data.drop(columns='otc', inplace=True)

In [68]:
stock_data.head()

,stock,open,high,low,close,volume,vwap,timestamp,transactions
0,SONY,77.15,77.680,76.39,76.65,660630.0,76.7866,2023-01-03 05:00:00,13809
1,SONY,77.99,78.390,77.49,77.80,596412.0,77.9470,2023-01-04 05:00:00,13577
2,SONY,77.78,78.115,77.31,77.50,540993.0,77.6747,2023-01-05 05:00:00,12203
3,SONY,79.23,81.640,79.10,81.39,896688.0,80.8119,2023-01-06 05:00:00,22789
4,SONY,81.68,83.340,81.63,82.62,918041.0,82.7863,2023-01-09 05:00:00,17973


In [69]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   stock         1500 non-null   object        
 1   open          1500 non-null   float64       
 2   high          1500 non-null   float64       
 3   low           1500 non-null   float64       
 4   close         1500 non-null   float64       
 5   volume        1500 non-null   float64       
 6   vwap          1500 non-null   float64       
 7   timestamp     1500 non-null   datetime64[ns]
 8   transactions  1500 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 105.6+ KB


# Load

In [70]:
#creating csv with the cleaned data for use in tableau
stock_data.to_csv('stock_data.csv')
portfolio_metrics.to_csv('portfolio_metrics.csv')
stock_metrics.to_csv('stock_metrics.csv')

In [71]:
#get environment variables
server = "DESKTOP"
database = "stocks_db"

#create the engine
engine = create_engine(f'mssql+pymssql://@{server}/{database}')

#write the dataframe to sql server table
stock_data.to_sql('stock_data', engine, if_exists='replace', index=False)
portfolio_metrics.to_sql('portfolio_metrics', engine, if_exists='replace', index=False)
stock_metrics.to_sql('stock_metrics', engine, if_exists='replace', index=False)

#close connection to engine
engine.dispose

<bound method Engine.dispose of Engine(mssql+pymssql://@DESKTOP/stocks_db)>